# 图片预处理

In [1]:
import os, shutil, platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle 

%matplotlib inline 

## 工具函数:判断操作系统种类

In [2]:
print("当前工作目录是:")
print(os.getcwd())

当前工作目录是:
/home/ubuntu/cat_vs_dog_cnn


In [3]:
from PIL import Image
print(Image.__file__)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/Image.py


In [4]:
def is_Windows_OS():
    if platform.system() == 'Windows':
        return True
    else:
        return False

## 定义的目录结构变量

In [5]:
# 训练集图片从zip包解压之后存放的目录
train_images_folder = 'train'

# 训练集图片目录
train_set_folder = 'train1'
train_set_folder_cat = 'train1/cats/'
train_set_folder_dog = 'train1/dogs/'

# 验证集图片目录
validation_set_folder = 'valid'
validation_set_folder_cat = 'valid/cats/'
validation_set_folder_dog = 'valid/dogs/'

# 测试集图片从zip包解压缩之后的存放目录
test_set_folder = 'test1'

## 新建有关目录结构

In [6]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir(train_set_folder)
os.mkdir(train_set_folder_cat)
os.mkdir(train_set_folder_dog)

rmrf_mkdir(validation_set_folder)
os.mkdir(validation_set_folder_cat)
os.mkdir(validation_set_folder_dog)

## 加载训练集目录

In [7]:
train_filenames = os.listdir(train_images_folder)
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

In [8]:
print (type(train_cat))
print (len(train_filenames))

<class 'filter'>
25000


In [9]:
print (train_filenames[24999])

cat.2814.jpg


定义一组移动有用的函数：

In [10]:
num_of_images_valid_set = 1500
        
def move_cat_images():
    timer = 0
    for filename in train_cat:
        if timer < num_of_images_valid_set:
            shutil.copy2('train/'+filename, validation_set_folder_cat)
        else:
            shutil.copy2('train/'+filename, train_set_folder_cat)
        timer = timer + 1

def move_dog_images():
    timer = 0
    for filename in train_dog:
        if timer < num_of_images_valid_set:
            shutil.copy2('train/'+filename, validation_set_folder_dog)
        else:
            shutil.copy2('train/'+filename, train_set_folder_dog)
        timer = timer + 1  

In [11]:
def fill_train_set_valid_set_folder_with_images():
    move_cat_images()
    move_dog_images()

In [12]:
fill_train_set_valid_set_folder_with_images()
print ('fill_train_set_valid_set_folder_with_images执行完毕')

fill_train_set_valid_set_folder_with_images执行完毕


# 搭建并且编译模型

In [17]:
from keras.applications.densenet import DenseNet169
from keras.models import Sequential, Model
from keras.layers import Conv2D, Input
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers import Flatten, Dropout
from keras.layers import Dense, GlobalAveragePooling2D


# 不包含原有模型的全连接层
dnesenet_model = DenseNet169(include_top=False, 
                             weights='imagenet',
                             input_shape = (224, 224, 3),
                             pooling='avg')

for layer in dnesenet_model.layers:
    layer.trainable = False

x = dnesenet_model.output

x = Dense(units = 1024, activation = 'relu',  name='fc1')(x)

x = Dropout(0.4, name='fc2_dropout')(x)

x = Dense(units = 128, activation = 'relu',  name='fc3')(x)

x = Dropout(0.4, name='fc4_dropout')(x)

x = Dense(units = 32, activation = 'relu',  name='fc5')(x)

x = Dropout(0.4, name='fc6_dropout')(x)

# Classifier
predictions = Dense(units = 1, activation='sigmoid',  name='predictions')(x)

model = Model(inputs=dnesenet_model.input, outputs=predictions)
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

## 训练模型

In [19]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64
target_size = (224, 224)

train_image_gen = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     rescale = 1./255,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)
print("训练集图片:")
training_set = train_image_gen.flow_from_directory('train1',
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 color_mode="rgb",
                                                 class_mode = 'binary')

valid_image_gen = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     rescale = 1./255,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)
print("验证集图片:")
validation_set = valid_image_gen.flow_from_directory('valid',
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 color_mode="rgb",
                                                 class_mode = 'binary')


训练集图片:
Found 22000 images belonging to 2 classes.
验证集图片:
Found 3000 images belonging to 2 classes.


## 开始训练模型

In [ ]:
# 训练集图片总量
train_size = 22000

# 每一轮训练需要执行的步长
steps_per_epoch = train_size/batch_size

# 验证集图片总量
valid_size = 3000
# 每一轮验证需要执行的步长
validation_steps = valid_size/batch_size

# 使用模型拟合图片数据
model.fit_generator(training_set, epochs = 5, steps_per_epoch = steps_per_epoch, 
                    validation_data=validation_set, validation_steps = validation_steps)

Epoch 1/5
344/343 [==============================] - 302s 879ms/step - loss: 0.0898 - acc: 0.9654 - val_loss: 0.3930 - val_acc: 0.8877
Epoch 3/5
 49/343 [===>..........................] - ETA: 3:10 - loss: 0.0781 - acc: 0.9726

## 开始预测

In [20]:
print("测试集图片:")
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory('test1',
                                                    target_size=target_size, 
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pred_result = model.predict(X_test, verbose=1)
pred_result = pred_result.clip(min=0.005, max=0.995)
print("预测执行完毕！")

FileNotFoundError: [Errno 2] No such file or directory: './train1/dogs/dog.410.jpg'

## 导出预测结果

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)